<a href="https://colab.research.google.com/github/alexledd/Blender-Colaboratory/blob/main/Simple_Blender_Render.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Form

blender_version = '3.5.1' #@param ['3.0.1','3.1.2','3.2.2' ,'3.3.8','3.4.1' , '3.5.1'] {allow-input: false}
project_name = 'DG_2' #@param {type: 'string'}
blend_file_path = '/content/DG_2.blend' #@param {type: 'string'}
#@markdown ---
animation = False #@param {type: 'boolean'}
start_frame =  1#@param {type: 'integer'}
end_frame =  250#@param {type: 'integer'}
#@markdown ---
gpu_enabled = True #@param {type:"boolean"}
optix_enabled = True #@param {type:"boolean"}
cpu_enabled = True #@param {type:"boolean"}

import os

!mkdir /content/$project_name
!mkdir /content/$project_name/rendered
!mkdir /content/$project_name/blend
!cp $blend_file_path /content/$project_name/blend

blend_file = os.path.basename(blend_file_path)

In [ ]:
#@title NVIDIA GPU Check
gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

if gpu[0] == "Tesla K80" and optix_enabled:
  print("OptiX disabled because of unsupported GPU")
  optix_enabled = False

In [ ]:
#@title Mount Google Drive (Optional)
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

In [ ]:
#@title Dependencies
# Add your own version of blender from https://ftp.nluug.nl/pub/graphics/blender/release
blender_url_dict = {'3.0.1'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.0/blender-3.0.1-linux-x64.tar.xz",
                    '3.1.2'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.1/blender-3.1.2-linux-x64.tar.xz",
                    '3.2.2'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.2/blender-3.2.2-linux-x64.tar.xz",
                    '3.3.8'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.3/blender-3.3.8-linux-x64.tar.xz",
                    '3.4.1'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.4/blender-3.4.1-linux-x64.tar.xz",
                    '3.5.1'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.5/blender-3.5.1-linux-x64.tar.xz"}

import os

blender_url = blender_url_dict[blender_version]
base_url = os.path.basename(blender_url)

!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1

In [ ]:
#@title Enable GPU rendering (Custom Properties add here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

renderer = "CUDA"
if optix_enabled:
    print("Note: You're currently using OptiX renderer. If an error occurred, set OptiX to False")
    renderer = "OPTIX"

if gpu_enabled:
  !cp /content/setgpu.py /content/$blender_version/

Note: You're currently using OptiX renderer. If an error occurred, set OptiX to False


In [ ]:
#@title Render
%cd /content/$blender_version
output_path = /content/$project_name/rendered

if animation:
    if start_frame == end_frame:
        !./$blender_version/blender -b '/content/$project_name/blend/{blend_file}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -a -- --cycles-device "{renderer}"
    else:
        !./$blender_version/blender -b '/content/$project_name/blend/{blend_file}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -s $start_frame -e $end_frame -a -- --cycles-device "{renderer}"
else:
    !./$blender_version/blender -b '/content/$project_name/blend/{blend_file}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -f $start_frame -- --cycles-device "{renderer}"